In [11]:
import pandas as pd
import numpy as np
import os
from pandas import Series, DataFrame

def open_files():
    directory = '/Users/piper/Piper Documents/R and Python/Project 2/DataRaw/Diabetes-Data/'
    with open ('/Users/piper/Piper Documents/R and Python/Project 2/DataRaw/Diabetes-Data/all_data.txt', 'w') as output_file:
        for data_file in os.listdir(directory):
            if data_file != '.DS_Store':
                print('reading from ' + data_file)
                input_file_path = os.path.join(directory, data_file)            
                with open (input_file_path) as input_file:
                    line = input_file.readline().strip()
                    while line:
                        output_line = line + '\t' + str(data_file) + '\n'
                        output_file.write(output_line)
                        line = input_file.readline().strip()
                    input_file.close()
        output_file.close()

if __name__ == "__main__":
    print('main')
    open_files()
    

main
reading from data-31
reading from data-36
reading from data-09
reading from data-07
reading from data-38
reading from data-53
reading from data-54
reading from data-62
reading from data-65
reading from data-06
reading from data-39
reading from data-01
reading from data-37
reading from data-08
reading from data-30
reading from data-64
reading from data-63
reading from data-55
reading from data-52
reading from data-70
reading from data-48
reading from data-41
reading from data-46
reading from data-12
reading from data-15
reading from data-23
reading from data-24
reading from data-47
reading from data-40
reading from data-49
reading from data-25
reading from data-22
reading from data-14
reading from data-13
reading from data-68
reading from data-57
reading from data-50
reading from data-59
reading from data-66
reading from data-61
reading from data-35
reading from data-32
reading from data-04
reading from data-03
reading from data-60
reading from data-58
reading from data-67
reading 